In [17]:
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_anthropic import ChatAnthropic
import os
from IPython.display import display, Markdown

In [18]:
class NaiveRAG:
    def __init__(self, collection_name: str):
        """
        ໂຫຼດ RAG ຈາກ Chroma DB ທີ່ມີຢູ່ແລ້ວ
        """
        # ໂຫຼດ ChromaDB client ຈາກ folder ທີ່ຕົນເອງສ້າງ
        self.client = chromadb.PersistentClient(path="../Vector/chroma_db")
        
        # ໂຫຼດ embedding model
        self.embedding_model = SentenceTransformer('D:/model/BAAI-bge-m3',device='cpu')
        
        # ໂຫຼດ collection ທີ່ມີຢູ່ແລ້ວ
        try:
            self.collection = self.client.get_collection(name=collection_name)
            print(f"✅ ໂຫຼດ collection ສໍາເລັດ: {collection_name}")
            print(f"   ມີເອກະສານທັງໝົດ: {self.collection.count()} ເອກະສານ")
        except Exception as e:
            print(f"❌ ບໍ່ສາມາດໂຫຼດ collection: {collection_name}")
            print(f"   ຂໍ້ຜິດພາດ: {e}")
            raise

        # ຕິດຕັ້ງ Anthropic Claude
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            raise ValueError("ກະລຸນາໃຫ້ ANTHROPIC_API_KEY")
        
        self.llm = ChatAnthropic(
            api_key=api_key,
            model="claude-opus-4-1-20250805",  
            temperature=0.0,
            max_tokens=2000
        )
    
    def search(self, query: str, n_results: int = 5):
        """
        ຄົ້ນຫາເອກະສານທີ່ກ່ຽວຂ້ອງ
        """
        query_embedding = self.embedding_model.encode([query]).tolist()
        
        results = self.collection.query(
            query_embeddings=query_embedding,
            n_results=n_results
        )
        
        docs = []
        for i in range(len(results['documents'][0])):
            docs.append({
                'text': results['documents'][0][i],
                'score': 1 - results['distances'][0][i],  # ປ່ຽນ distance ເປັນ similarity
                'id': results['ids'][0][i]
            })
        
        return docs
    
    def ask(self, question: str, n_results: int = 5):
        """
        ຖາມຄໍາຖາມ ແລະ ໄດ້ຄໍາຕອບ
        """
        print(f"🔍 ກໍາລັງຄົ້ນຫາ: {question}")
        
        # 1. ຄົ້ນຫາເອກະສານທີ່ກ່ຽວຂ້ອງ
        docs = self.search(question, n_results)
        
        if not docs:
            return "❌ ບໍ່ພົບເອກະສານທີ່ກ່ຽວຂ້ອງ"
        
        print(f"📄 ພົບເອກະສານ: {len(docs)} ເອກະສານ")
        
        # 2. ສ້າງ context
        context = "\n\n".join([doc['text'] for doc in docs])
        
        # 3. ສ້າງ prompt
        prompt = f"""ທ່ານເປັນ AI Assistant ທີ່ຊ່ຽວຊານໃນການຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາ.

        ຄຳແນະນຳ:
        1. ຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາເທົ່ານັ້ນ
        2. ຖ້າບໍ່ພົບຄຳຕອບໃນເອກະສານ, ໃຫ້ບອກວ່າບໍ່ພົບຂໍ້ມູນທີ່ກ່ຽວຂ້ອງ
        3. ລະບຸແຫຼ່ງຂໍ້ມູນທີ່ໃຊ້ໃນການຕອບ
        4. ຕອບເປັນພາສາລາວເທົ່ານັ້ນ, ໃຫ້ຄຳຕອບທີ່ຊັດເຈນ ແລະ ລະອຽດ
        5. ຕອບໃຫ້ເປັນ Format markdown
                
        ຂໍ້ມູນອ້າງອີງ:
        {context}

        ຄໍາຖາມ: {question}
        """
        
        # 4. ສ້າງຄໍາຕອບດ້ວຍ Claude
        try:
            response = self.llm.invoke(prompt)
            answer = response.content.strip()
            
            return {
                'question': question,
                'answer': answer,
                'sources': docs
            }
            
        except Exception as e:
            return f"❌ ເກີດຂໍ້ຜິດພາດ: {str(e)}"

In [19]:
# ວິທີໃຊ້ງານງ່າຍໆ
def main():
    # ໃສ່ຊື່ collection ຂອງທ່ານ
    COLLECTION_NAME = "pdf_documents"  # ປ່ຽນຊື່ນີ້ຕາມ collection vector ທີ່ຕົນເອງສ້າງ
    
    try:
        # ສ້າງ RAG
        rag = NaiveRAG(
            collection_name=COLLECTION_NAME, 
        )
        
        # ຖາມຄໍາຖາມ
        while True:
            question = input("\n❓ ຖາມຄໍາຖາມ (ຫຼື 'quit' ເພື່ອອອກ): ")
            
            if question.lower() in ['quit', 'exit', 'ອອກ']:
                print("👋 ລາກ່ອນ!")
                break
            
            if question.strip():
                print("\n🤖 ກໍາລັງປະມວນຜົນ...")
                # n_result ສາມາດປັບໄດ້ຕາມໃຈ ແຕ່ເນື່ອງຈາກວ່າ model ເຮົາມັນຈຳກັດ token ສະນັ້ນແມ່ນໃຫ້ຮະວັງ ເລື່ອງຂອງ context window ນຳເດີ ອາດຈະເຮັດໃຫ້ error ໄດ້. ແຕ່ຖ້າຢາກທົດລອງດ້ວຍຜົນລັບຈຳນວນຫລາຍແມ່ນແນະນຳໃຫ້ໃຊ້ model LLM ເສຍເງິນ
                result = rag.ask(question=question,n_results=20)
                
                if isinstance(result, dict):
                    display(Markdown(f"\n✅ ຄໍາຕອບ: {result['answer']}"))  
                else:
                    print(f"\n{result}")
    
    except Exception as e:
        print(f"❌ ເກີດຂໍ້ຜິດພາດ: {e}")

In [20]:
if __name__ == "__main__":
    main()
    

✅ ໂຫຼດ collection ສໍາເລັດ: pdf_documents
   ມີເອກະສານທັງໝົດ: 95 ເອກະສານ

🤖 ກໍາລັງປະມວນຜົນ...
🔍 ກໍາລັງຄົ້ນຫາ: ອັດຕາດອກ  ເບ້ຍ ຕໍ່ປິ fixed loan ເທົ່າໃດ
📄 ພົບເອກະສານ: 20 ເອກະສານ



✅ ຄໍາຕອບ: ອີງຕາມເອກະສານທີ່ໃຫ້ມາ, ອັດຕາດອກເບ້ຍຕໍ່ປີຂອງເງິນກູ້ຄົງທີ່ (Fixed Loan) ມີດັ່ງນີ້:

## ອັດຕາດອກເບ້ຍເງິນກູ້ຄົງທີ່ (Fixed Loan)

- **ສະກຸນເງິນກີບ:** 9% - 10.25% ຕໍ່ປີ
- **ສະກຸນເງິນໂດລາ/ບາດ:** 7.50% - 9.25% ຕໍ່ປີ

### ຄ່າທຳນຽມເພີ່ມເຕີມ:
- ຄ່າທຳນຽມປະເມີນຫຼັກຊັບ: 0.2% ຂອງວົງເງິນອະນຸມັດ (ສູງສຸດບໍ່ເກີນ 5 ລ້ານກີບ)
- ຄ່າທຳນຽມອະນຸມັດ: 0.5 - 1% ຂອງວົງເງິນອະນຸມັດ

**ໝາຍເຫດ:** ອັດຕາດອກເບ້ຍ ແລະ ຄ່າທຳນຽມ ອາດຈະປ່ຽນແປງຕາມການປະກາດໃຊ້ໃນແຕ່ລະໄລຍະ

*ແຫຼ່ງຂໍ້ມູນ: ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ້າທີ 52*


🤖 ກໍາລັງປະມວນຜົນ...
🔍 ກໍາລັງຄົ້ນຫາ: ບໍລິການຮັບເງິນສົດກັບທີັ່ ຄ່າທໍານຽມເທົ່າໃດ
📄 ພົບເອກະສານ: 20 ເອກະສານ



✅ ຄໍາຕອບ: ອີງຕາມເອກະສານທີ່ໃຫ້ມາ, **ຄ່າທໍານຽມບໍລິການຮັບເງິນສົດກັບທີ່** ມີດັ່ງນີ້:

## ຄ່າທໍານຽມຕາມໄລຍະທາງ:

- **ໄລຍະທາງ 10 KM ລົງມາ:** ເກັບ 200,000 ກີບ/ຄັ້ງ
- **ໄລຍະທາງ 10.1 KM - 15 KM:** ເກັບ 300,000 ກີບ/ຄັ້ງ  
- **ໄລຍະທາງ 15.1 KM - 20 KM:** ເກັບ 400,000 ກີບ/ຄັ້ງ

## ໝາຍເຫດ:
ຄ່າທໍານຽມແມ່ນອີງຕາມແຈ້ງການຄ່າທໍານຽມທີ່ ທຄຕລ ປະກາດນໍາໃຊ້ແຕ່ລະໄລຍະ

---
**ແຫຼ່ງຂໍ້ມູນ:** ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ້າທີ 41

👋 ລາກ່ອນ!
